In [15]:
# !pip install -r ./requirements.txt

In [15]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_experiments_for_inprocessors

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_experiments_for_inprocessors

In [5]:
!pip uninstall starlette -y

Found existing installation: starlette 0.32.0.post1
Uninstalling starlette-0.32.0.post1:
  Successfully uninstalled starlette-0.32.0.post1


In [7]:
# !pip install gast

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [4]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import StudentPerformancePortugueseDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS

from source.experiment_interface import run_exp_iter_with_inprocessor

pip install 'aif360[LawSchoolGPA]'


## Define Input Variables

In [5]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'EGR_student_performance'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIRNESS_INTERVENTION_NAME = 'EGR'
FAIR_INTERVENTION_PARAMS_LST = ['DemographicParity']
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                     FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, 'student_performance_por_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': '65f39387-1ed2-4e6d-9670-ae8c552a0f65',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  65f39387-1ed2-4e6d-9670-ae8c552a0f65


## Initialize custom objects

In [9]:
data_loader = StudentPerformancePortugueseDataset()
data_loader.X_data.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
0,18,4,4,2,2,0,4,3,4,1,...,course,mother,yes,no,no,no,yes,yes,no,no
1,17,1,1,1,2,0,5,3,3,1,...,course,father,no,yes,no,no,no,yes,yes,no
2,15,1,1,1,2,0,4,3,2,2,...,other,mother,yes,no,no,no,yes,yes,yes,no
3,15,4,2,1,3,0,3,2,2,1,...,home,mother,no,yes,no,yes,yes,yes,yes,yes
4,16,3,3,1,2,0,4,3,2,1,...,home,father,no,yes,no,no,yes,yes,no,no


In [10]:
data_loader.X_data.shape

(649, 32)

## Run experiment iterations

### Experiment iteration 1

In [11]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='StudentPerformancePortugueseDataset',
                              inprocessor_name='ExponentiatedGradientReduction',
                              verbose=True)

2024-01-09 20:23:20 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': "['DemographicParity']",
 'model_init_seed': 100,
 'session_uuid': '65f39387-1ed2-4e6d-9670-ae8c552a0f65'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-09 20:23:20 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  DemographicParity
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__rom

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [13]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [14]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='StudentPerformancePortugueseDataset',
                              inprocessor_name='ExponentiatedGradientReduction',
                              verbose=True)

2024-01-09 20:25:29 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': "['DemographicParity']",
 'intervention_param': 'DemographicParity',
 'model_init_seed': 200,
 'session_uuid': '65f39387-1ed2-4e6d-9670-ae8c552a0f65'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-09 20:25:29 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  DemographicParity
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__rom

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [15]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [16]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='StudentPerformancePortugueseDataset',
                              inprocessor_name='ExponentiatedGradientReduction',
                              verbose=True)

2024-01-09 20:27:36 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': "['DemographicParity']",
 'intervention_param': 'DemographicParity',
 'model_init_seed': 300,
 'session_uuid': '65f39387-1ed2-4e6d-9670-ae8c552a0f65'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-09 20:27:36 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  DemographicParity
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__rom

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [17]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [18]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='StudentPerformancePortugueseDataset',
                              inprocessor_name='ExponentiatedGradientReduction',
                              verbose=True)

2024-01-09 20:29:40 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': "['DemographicParity']",
 'intervention_param': 'DemographicParity',
 'model_init_seed': 400,
 'session_uuid': '65f39387-1ed2-4e6d-9670-ae8c552a0f65'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-09 20:29:40 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  DemographicParity
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__rom

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 5

In [19]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [20]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='StudentPerformancePortugueseDataset',
                              inprocessor_name='ExponentiatedGradientReduction',
                              verbose=True)

2024-01-09 20:31:44 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'experiment_iteration': 'Exp_iter_5',
 'fair_intervention_params_lst': "['DemographicParity']",
 'intervention_param': 'DemographicParity',
 'model_init_seed': 500,
 'session_uuid': '65f39387-1ed2-4e6d-9670-ae8c552a0f65'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-09 20:31:44 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  DemographicParity
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__rom

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 6

In [21]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [22]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='StudentPerformancePortugueseDataset',
                              inprocessor_name='ExponentiatedGradientReduction',
                              verbose=True)

2024-01-09 20:33:56 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': "['DemographicParity']",
 'intervention_param': 'DemographicParity',
 'model_init_seed': 600,
 'session_uuid': '65f39387-1ed2-4e6d-9670-ae8c552a0f65'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-09 20:33:56 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  DemographicParity
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__rom

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]